<a href="https://colab.research.google.com/github/Theanh130124/RAG_Finance/blob/master/rag_finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pwd

'/content'

In [ ]:
!pip install pi-heif "unstructured[local-inference,pdf]" pdfminer.six pypdf Pillow
!pip install -U langchain-community langchain-core
!pip install -U langchain-text-splitters
!pip install langchain==0.3.27
!pip install qdrant-client
!pip install unstructured
!pip install python-docx
!pip install langchain-qdrant qdrant-client




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 43.4 MB/s eta 0:00:00
   

#Extract PDF


In [ ]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


def load_pdf_files(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.pdf",
        loader_cls=UnstructuredPDFLoader
    )
    documents = loader.load()
    return documents

#Extract .txt

In [ ]:
from langchain.document_loaders import DirectoryLoader , UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_word_files(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.docx",
        loader_cls=UnstructuredWordDocumentLoader #loại file cần load
    )
    documents = loader.load()
    return documents

In [ ]:
extracted_data = load_word_files(data='DataChatbot')

In [ ]:
extracted_data

#Preprocessing (Tiền xử lý dữ liệu)

In [ ]:
import re

def preprocess_data(text):


    #Xóa URL
    text = re.sub(r'(https?://\S+|www\.\S+)', '', text)
    # Xoá các dòng chỉ chứa dấu = hoặc -
    text = re.sub(r'^[=\-]{2,}\s*$', '', text, flags=re.MULTILINE)
    #Xoá các ký tự bảng markdown (|, ---)
    text = re.sub(r'\|.*?\|', '', text)
    # Xoá emoji và ký tự Unicode không cần thiết
    text = re.sub(r'[^\w\s,.!?à-ỹÀ-Ỹ\-–]', '', text)
    # Xoá khoảng trắng thừa và dòng trống
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
from langchain_core.documents import Document

cleaned_data = []
for doc in extracted_data:
    cleaned_content = preprocess_data(doc.page_content)
    cleaned_doc = Document(
        page_content=cleaned_content,
        metadata=doc.metadata
    )
    cleaned_data.append(cleaned_doc)

In [ ]:
cleaned_data

#Chunking với text

In [ ]:
def text_split(cleaned_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=20,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    text_chunks = text_splitter.split_documents(cleaned_data)
    return text_chunks
